In [1]:
import pandas as pd

from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split


In [2]:
import mlflow
from mlflow.models import infer_signature

In [3]:
## SET THE TRACKING URI

mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

In [4]:
## LOAD THE DATASET
X,y= datasets.load_iris(return_X_y=True)

## SPLIT THE DATA INTO TRAINING  AND TEST DATA SETS
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20)



In [5]:
## DEFINE THE MODEL HYPER PARAMETERs


params = {"penalty":"l2","solver": "lbfgs", "max_iter": 1000, "multi_class": "auto", "random_state": 8888}

In [6]:
## Train the Model

lr = LogisticRegression(**params)

lr.fit(X_train,y_train)

d:\My_Work\UdeMy\3. Complete DS,ML,DL,NLP- BOOTCAMP\My_MLFlow\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888)

In [7]:
X_test

array([[5.8, 2.8, 5.1, 2.4],
       [6.3, 2.7, 4.9, 1.8],
       [7.7, 3. , 6.1, 2.3],
       [4.3, 3. , 1.1, 0.1],
       [5. , 3.2, 1.2, 0.2],
       [5.8, 2.7, 5.1, 1.9],
       [4.7, 3.2, 1.3, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.4, 3. , 1.3, 0.2],
       [6.3, 3.3, 4.7, 1.6],
       [5.4, 3. , 4.5, 1.5],
       [6.5, 3. , 5.5, 1.8],
       [5.1, 3.8, 1.5, 0.3],
       [6.1, 2.8, 4.7, 1.2],
       [6.3, 2.3, 4.4, 1.3],
       [7.6, 3. , 6.6, 2.1],
       [5.7, 2.8, 4.5, 1.3],
       [5.6, 3. , 4.1, 1.3],
       [6.2, 2.9, 4.3, 1.3],
       [6. , 2.2, 4. , 1. ],
       [6.7, 3.1, 4.4, 1.4],
       [6.1, 2.8, 4. , 1.3],
       [6.9, 3.1, 4.9, 1.5],
       [5.9, 3. , 4.2, 1.5],
       [6.2, 3.4, 5.4, 2.3],
       [6.8, 2.8, 4.8, 1.4],
       [5.3, 3.7, 1.5, 0.2],
       [6.3, 2.8, 5.1, 1.5],
       [5.7, 4.4, 1.5, 0.4],
       [5. , 3.4, 1.5, 0.2]])

In [8]:
## Prediction on the test set
y_pred=lr.predict(X_test)
y_pred

array([2, 2, 2, 0, 0, 2, 0, 0, 0, 1, 1, 2, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 0, 2, 0, 0])

In [10]:
accuracy=accuracy_score(y_test,y_pred)
print(accuracy)

1.0


### MLFLOW Tracking

In [13]:
## Create a new MLFLOW experiment

mlflow.set_experiment("MLFLOW Quickstart By Vishal")

2025/06/09 12:53:31 INFO mlflow.tracking.fluent: Experiment with name 'MLFLOW Quickstart By Vishal' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/897524100848639810', creation_time=1749453811188, experiment_id='897524100848639810', last_update_time=1749453811188, lifecycle_stage='active', name='MLFLOW Quickstart By Vishal', tags={}>

In [14]:
## Start an MLFLOW run

with mlflow.start_run():
    
    ## log the hyperparameters
    mlflow.log_params(params)

    ## Log the accuracy metrics
    mlflow.log_metric("accuracy",accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for Iris Data")

    ## Infer the Model Signature

    signature=infer_signature(X_train, lr.predict(X_train))

    ## log the model
    model_info=mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="Tracking-Quickstart-by-Vishal",

    )

Successfully registered model 'Tracking-Quickstart-by-Vishal'.
2025/06/09 12:59:18 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Tracking-Quickstart-by-Vishal, version 1


🏃 View run polite-toad-1 at: http://127.0.0.1:5000/#/experiments/897524100848639810/runs/4cc4e73a140d4b8db8a5174bfddf585b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/897524100848639810


Created version '1' of model 'Tracking-Quickstart-by-Vishal'.


In [15]:
model_info.model_uri

'runs:/4cc4e73a140d4b8db8a5174bfddf585b/iris_model'

## Inferencing And Validating Model

In [27]:
from mlflow.models import validate_serving_input

import json

model_uri = model_info.model_uri

# The model is logged with an input example. MLflow converts
# it into the serving payload format for the deployed model endpoint,
# and saves it to 'serving_input_payload.json'


# Define artifact path
artifact_path = "mlflow-artifacts:/897524100848639810/4cc4e73a140d4b8db8a5174bfddf585b/artifacts/iris_model/serving_input_example.json"

# Step 1: Download the artifact locally
local_path = mlflow.artifacts.download_artifacts(artifact_path)

# Step 2: Load the JSON content
with open(local_path, "r") as f:
    serving_payload = json.load(f)

# Step 3: Validate the payload
validate_serving_input(model_uri, serving_payload)

array([2, 1, 0, 0, 1, 0, 2, 1, 0, 1, 0, 2, 2, 2, 0, 2, 0, 1, 2, 0, 2, 0,
       2, 0, 1, 2, 0, 2, 1, 1, 1, 0, 1, 0, 2, 1, 2, 0, 0, 2, 2, 2, 2, 1,
       2, 0, 1, 1, 2, 2, 2, 2, 1, 1, 0, 1, 1, 1, 0, 2, 0, 2, 0, 1, 1, 1,
       1, 1, 2, 2, 1, 2, 0, 1, 1, 2, 2, 0, 0, 0, 0, 0, 2, 2, 1, 1, 0, 2,
       0, 1, 2, 2, 0, 0, 0, 2, 1, 2, 0, 0, 0, 0, 1, 2, 2, 1, 0, 2, 0, 2,
       1, 1, 0, 2, 2, 0, 0, 2, 0, 2])

## Load the Model Back for Prediction as a Generic Python Function Model

In [ ]:
loaded_model=mlflow.pyfunc.load_model(model_info.model_uri)

predictions=loaded_model.predict(X_test)

iris_features_name=datasets.load_iris().feature_names

result=pd.DataFrame(X_test,columns=iris_features_name)

result["Actual_Class"]=y_test

result["Predicted_Class"]=predictions

In [34]:
result

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Actual_Class,Predicted_Class
0,5.8,2.8,5.1,2.4,2,2
1,6.3,2.7,4.9,1.8,2,2
2,7.7,3.0,6.1,2.3,2,2
3,4.3,3.0,1.1,0.1,0,0
4,5.0,3.2,1.2,0.2,0,0
5,5.8,2.7,5.1,1.9,2,2
6,4.7,3.2,1.3,0.2,0,0
7,4.4,2.9,1.4,0.2,0,0
8,4.4,3.0,1.3,0.2,0,0
9,6.3,3.3,4.7,1.6,1,1


### Model Registry

The MLflow Model Registry component is a centralized model store, set of APIs, and UI, to collaboratively manage the full lifecycle of an MLflow Model.

It provides model lineage (which MLflow experiment and run produced the model), model versioning, model aliasing, model tagging, and annotations.

In [35]:
## log the model
model_info=mlflow.sklearn.log_model(
    sk_model=lr,
    artifact_path="iris_model",
    signature=signature,
    input_example=X_train,
    # registered_model_name="Tracking-Quickstart-by-Vishal"    (Skipping Registering The model, First Validating the MOdel before Registering )
)

## Create a new MLFLOW experiment

In [39]:

mlflow.set_experiment("MLFLOW Quickstart By Vishal")


# End any old active run to avoid errors
# if mlflow.active_run():
#     mlflow.end_run()

    
## Start an MLFLOW run

with mlflow.start_run():

    ## log the hyperparameters
    mlflow.log_params(params)

    ## Log the accuracy metrics
    mlflow.log_metric("accuracy",0.85)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info2", "Basic LR model for iris data")

    ## Infer the model signature

    signature=infer_signature(X_train,lr.predict(X_train))

    ## log the model

    model_info=model_info

🏃 View run ambitious-stag-437 at: http://127.0.0.1:5000/#/experiments/897524100848639810/runs/cf428510fd5a400791bf7b614c47e78f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/897524100848639810


## Inferencing the model from Model Registry

In [40]:

import mlflow.sklearn

model_name="Tracking-Quickstart-by-Vishal"
model_version="latest"

model_uri=f"models:/{model_name}/{model_version}"

model=mlflow.sklearn.load_model(model_uri)
model

LogisticRegression(max_iter=1000, multi_class='auto', random_state=8888)

In [41]:
model_uri

'models:/Tracking-Quickstart-by-Vishal/latest'

In [42]:
y_pred_new=model.predict(X_test)
y_pred_new

array([2, 2, 2, 0, 0, 2, 0, 0, 0, 1, 1, 2, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 0, 2, 0, 0])